In [31]:
from colbert.data import Queries
from colbert.infra import Run, RunConfig, ColBERTConfig
from colbert.infra import Run, RunConfig, ColBERTConfig
from colbert import Indexer
from colbert import Searcher
import os
# import cld3
import pandas as pd

In [5]:
eval_df = pd.read_csv("/raid/nlp/sameer/mahGRs/finance_questions.csv")
eval_df = eval_df[eval_df['Documents'].notna()]
print(len(eval_df))

34


BM25 Sparse retriever

In [2]:
doc_sections = os.listdir("/raid/nlp/sameer/mahGRs/GRs")
doc_sections = ['Finance_Department']
doc_freq = {}

corpus = []
corpus2id = {}
for section in doc_sections:
    doc_freq[section] = len(os.listdir("/raid/nlp/sameer/mahGRs/GRs/" + section))
    for doc in os.listdir("/raid/nlp/sameer/mahGRs/GRs/" + section):
        if doc.endswith("mr.txt"):
            f = open("/raid/nlp/sameer/mahGRs/GRs/"+ section + "/" + doc, "r")
            corpus.append(f.read().replace("\n"," "))
            corpus2id[corpus[-1]] = doc

# print(cld3.get_language(corpus[1]))
# corpus = [doc.replace("\n"," ") for doc in corpus if cld3.get_language(doc)[0]=="mr" and cld3.get_language(doc)[1]>0.9 and cld3.get_language(doc)[-1]>0.8]

In [4]:
print(corpus[443]), print(corpus2id[corpus[443]])

# Page 1 स्वच्छ भारत अभियान (नागरी) अंतर्गत नागरी घनकचरा व्यवस्थापन प्रकल्पास प्रशासकीय मान्यता देण्याबाबत. माहूर नगरपंचायत, जिल्हा नांदेड. महाराष्ट्र शासन नगर विकास विभाग शासन निर्णय क्रमांक:- स्वमअ-२०१९/प्र.क्र.८६ (६६) /नवि-३४ हुतात्मा राजगुरू चौक, मादाम कामा मार्ग मंत्रालय, मुंबई-४०० ०३२. दिनांक: १२ जुलै, २०१९ वाचा: १) शासन निर्णय, नगर विकास विभाग, क्रमांक: स्वभाअ-२०१५/प्र. क्र. २३/नवि-३४, दि.१५.०५.२०१५ २) स्वच्छ भारत अभियान (नागरी) अंतर्गत गठीत उच्चाधिकार समितीच्या दिनांक १ मार्च २०१९ रोजी पार पडलेल्या बैठकीचे इतिवृत्त. प्रस्तावना : देशातील सर्व शहरांमधील नागरिकांना स्वच्छ पर्यावरण व चांगले आरोग्य मिळावे, या करिता या शहरामधील सर्व नागरीकांना शौचालयांची सुविधा उपलब्ध करून देण्यासाठी केंद्र शासनामार्फत “स्वच्छ भारत अभियान (नागरी)” संपूर्ण देशामध्ये राबविण्यात येत आहे. केंद्र शासनाच्या “स्वच्छ भारत अभियान (नागरी)” च्या धर्तीवर संदर्भीय क्रमांक १ येथील शासन निर्णयान्वये राज्यामध्ये “स्वच्छ महाराष्ट्र अभियान (नागरी)” ची अंमलबजावणी सुरू आहे. या अभियानांतर्गत शहरे “हागणदारी मुक्त” करणे व 

(None, None)

In [3]:
from rank_bm25 import BM25Okapi

# corpus = [doc.replace("\n"," ") for doc in corpus if cld3.get_language(doc)[0]=="mr" and cld3.get_language(doc)[1]>0.9 and cld3.get_language(doc)[-1]>0.8]
tokenized_corpus = [doc.split(" ") for doc in corpus]
bm25 = BM25Okapi(tokenized_corpus)

In [6]:
for row in eval_df[:1].iterrows():
    print(row[1])

S No                                                                1
Question            १ जानेवारी २०१९ पासून राज्य शासनाच्या कर्मचाऱ्...
English Question    What is the increase in dearness allowance app...
English Answer      The government resolution does not provide inf...
Marathi Answer      सरकारी ठराव 1 जानेवारी 2019 पासून राज्य सरकारी...
Documents           202203301441163905.pdf 202101141237329905.pdf ...
Comment                                                           NaN
Name: 0, dtype: object


In [7]:
import math

# f_results = open("results_bm25.txt", "a")
full_questions = list(eval_df['Question'])
gold_docs = list(eval_df['Documents'])
cnt = 0
invalid_cnt = 0
for i in range(0,len(eval_df)):
    query = full_questions[i].replace("\n", " ")
    tokenized_query = query.split(" ")
    # try:
    # gold = gold_docs[i].split()[0].split(".")[0]
    gold = set([doc_name.split(".")[0] for doc_name in set(gold_docs[i].split())])
    bm25_docs = bm25.get_top_n(tokenized_query, corpus, n=10)
    retrieval = set()
    for doc in bm25_docs:
        retrieval.add(corpus2id[doc].split(".")[0])
        # if len(retrieval.intersection(gold)) == len(gold):
        #     cnt += 1
    # if gold in retrieval:
    if len(gold.intersection(retrieval)):
        cnt += 1
print(invalid_cnt)
print((cnt)/(len(eval_df)-invalid_cnt))

0
0.9411764705882353


ColBERT Dense retriever

In [24]:
with Run().context(RunConfig(nranks=1, experiment="marathi-synthetic")):

        config = ColBERTConfig(
            nbits=2,
            root="/raid/nlp/sameer/mahGRs/ColBERT/",
        )
        indexer = Indexer(checkpoint="/raid/nlp/sameer/mahGRs/ColBERT/experiments/marathi-synthetic/none/2024-05/12/11.40.07/checkpoints/colbert", config=config)
        indexer.index(name="msmarco.nbits=2_marathi_custom", collection="/raid/nlp/sameer/mahGRs/index_marathi_finance.tsv")



[May 18, 11:34:25] #> Creating directory /raid/nlp/sameer/mahGRs/ColBERT/experiments/marathi-synthetic/indexes/msmarco.nbits=2_marathi_custom 


#> Starting...
nranks = 1 	 num_gpus = 8 	 device=0
{
    "query_token_id": "[unused0]",
    "doc_token_id": "[unused1]",
    "query_token": "[Q]",
    "doc_token": "[D]",
    "ncells": null,
    "centroid_score_threshold": null,
    "ndocs": null,
    "load_index_with_mmap": false,
    "index_path": null,
    "nbits": 2,
    "kmeans_niters": 4,
    "resume": false,
    "similarity": "cosine",
    "bsize": 64,
    "accumsteps": 1,
    "lr": 3e-6,
    "maxsteps": 350,
    "save_every": null,
    "warmup": null,
    "warmup_bert": null,
    "relu": false,
    "nway": 2,
    "use_ib_negatives": false,
    "reranker": false,
    "distillation_alpha": 1.0,
    "ignore_scores": false,
    "model_name": null,
    "query_maxlen": 32,
    "attend_to_mask_tokens": false,
    "interaction": "colbert",
    "dim": 128,
    "doc_maxlen": 220,
    "mask_pu

WARNING clustering 112427 points to 4096 centroids: please provide at least 159744 training points


  Iteration 3 (0.38 s, search 0.04 s): objective=14797.8 imbalance=1.483 nsplit=0        
[May 18, 11:36:48] Loading decompress_residuals_cpp extension (set COLBERT_LOAD_TORCH_EXTENSION_VERBOSE=True for more info)...
[May 18, 11:36:48] Loading packbits_cpp extension (set COLBERT_LOAD_TORCH_EXTENSION_VERBOSE=True for more info)...
[0.023, 0.024, 0.022, 0.027, 0.022, 0.022, 0.024, 0.024, 0.024, 0.023, 0.022, 0.023, 0.025, 0.021, 0.023, 0.023, 0.024, 0.025, 0.019, 0.022, 0.023, 0.023, 0.021, 0.024, 0.022, 0.025, 0.026, 0.025, 0.023, 0.025, 0.023, 0.026, 0.022, 0.02, 0.023, 0.022, 0.025, 0.022, 0.024, 0.023, 0.023, 0.026, 0.024, 0.023, 0.022, 0.023, 0.024, 0.023, 0.023, 0.026, 0.025, 0.023, 0.025, 0.022, 0.024, 0.024, 0.021, 0.025, 0.023, 0.022, 0.022, 0.021, 0.026, 0.022, 0.022, 0.025, 0.026, 0.022, 0.023, 0.023, 0.023, 0.025, 0.023, 0.022, 0.024, 0.022, 0.024, 0.023, 0.022, 0.023, 0.025, 0.023, 0.023, 0.022, 0.023, 0.023, 0.022, 0.024, 0.023, 0.023, 0.02, 0.022, 0.024, 0.025, 0.023, 0.02

0it [00:00, ?it/s]

[May 18, 11:36:54] [0] 		 #> Saving chunk 0: 	 609 passages and 118,344 embeddings. From #0 onward.


1it [00:05,  5.72s/it]
100%|██████████| 4096/4096 [00:00<00:00, 122628.39it/s]


[May 18, 11:36:56] [0] 		 #> Checking all files were saved...
[May 18, 11:36:56] [0] 		 Found all files!
[May 18, 11:36:56] [0] 		 #> Building IVF...
[May 18, 11:36:56] [0] 		 #> Loading codes...
[May 18, 11:36:56] [0] 		 Sorting codes...
[May 18, 11:36:56] [0] 		 Getting unique codes...
[May 18, 11:36:56] #> Optimizing IVF to store map from centroids to list of pids..
[May 18, 11:36:56] #> Building the emb2pid mapping..
[May 18, 11:36:56] len(emb2pid) = 118344
[May 18, 11:36:56] #> Saved optimized IVF to /raid/nlp/sameer/mahGRs/ColBERT/experiments/marathi-synthetic/indexes/msmarco.nbits=2_marathi_custom/ivf.pid.pt
[May 18, 11:36:56] [0] 		 #> Saving the indexing metadata to /raid/nlp/sameer/mahGRs/ColBERT/experiments/marathi-synthetic/indexes/msmarco.nbits=2_marathi_custom/metadata.json ..
#> Joined...


In [36]:
with Run().context(RunConfig(nranks=1, experiment="marathi-synthetic")):

        config = ColBERTConfig(
            root="/path/to/experiments",
        )
        searcher = Searcher(index="msmarco.nbits=2_marathi_custom", config=config)
        queries = Queries("/raid/nlp/sameer/mahGRs/queries_marathi.tsv")
        ranking = searcher.search_all(queries, k=10)
        ranks = ranking.flat_ranking
        print(ranks)

[May 18, 11:47:34] #> Loading collection...
0M 
[May 18, 11:47:37] #> Loading codec...
[May 18, 11:47:37] #> Loading IVF...
[May 18, 11:47:37] #> Loading doclens...


100%|██████████| 1/1 [00:00<00:00, 3775.25it/s]

[May 18, 11:47:37] #> Loading codes and residuals...



100%|██████████| 1/1 [00:00<00:00, 57.88it/s]

[May 18, 11:47:37] #> Loading the queries from /raid/nlp/sameer/mahGRs/queries_marathi.tsv ...
[May 18, 11:47:37] #> Got 34 queries. All QIDs are unique.




#> QueryTokenizer.tensorize(batch_text[0], batch_background[0], bsize) ==
#> Input: . १ जानेवारी २०१९ पासून राज्य शासनाच्या कर्मचाऱ्यांना लागू असलेल्या महागाई भत्यात किती वाढ झाली आहे?, 		 True, 		 None
#> Output IDs: torch.Size([32]), tensor([  101,     1,   100,  1319, 29876, 29863, 29871, 29876, 29869, 29878,
          100,  1328, 29876, 29874, 29863,  1333, 29876, 29855, 29868,  1336,
        29876, 29874, 29863, 29876, 29854, 29868, 29876,  1315, 29869, 29867,
        29854,   102], device='cuda:0')
#> Output Mask: torch.Size([32]), tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1], device='cuda:0')



34it [00:02, 14.84it/s]

[(0, 475, 1, 21.953125), (0, 315, 2, 21.8125), (0, 346, 3, 21.796875), (0, 278, 4, 21.796875), (0, 367, 5, 21.734375), (0, 497, 6, 21.640625), (0, 134, 7, 21.609375), (0, 582, 8, 21.609375), (0, 87, 9, 21.4375), (0, 502, 10, 21.375), (1, 183, 1, 19.5625), (1, 250, 2, 18.671875), (1, 256, 3, 18.328125), (1, 291, 4, 18.171875), (1, 349, 5, 18.078125), (1, 325, 6, 18.078125), (1, 335, 7, 17.890625), (1, 115, 8, 17.859375), (1, 460, 9, 17.84375), (1, 135, 10, 17.84375), (2, 504, 1, 21.671875), (2, 235, 2, 21.578125), (2, 372, 3, 21.578125), (2, 61, 4, 20.65625), (2, 352, 5, 20.390625), (2, 229, 6, 20.359375), (2, 455, 7, 20.296875), (2, 234, 8, 18.96875), (2, 211, 9, 18.90625), (2, 553, 10, 18.875), (3, 508, 1, 23.90625), (3, 278, 2, 23.15625), (3, 346, 3, 23.078125), (3, 384, 4, 22.9375), (3, 187, 5, 22.84375), (3, 542, 6, 22.8125), (3, 502, 7, 22.796875), (3, 289, 8, 22.765625), (3, 493, 9, 22.765625), (3, 92, 10, 22.640625), (4, 508, 1, 23.90625), (4, 278, 2, 23.15625), (4, 346, 3, 23.0

In [ ]:
len(corpus), len(corpus2id)

(609, 609)

Evaluation

In [37]:
query2doc = {}
for tup in ranks:
    if tup[0] not in query2doc:
        query2doc[tup[0]] = []
    else:
        query2doc[tup[0]].append(corpus2id[corpus[tup[1]]])
# for tup in ranks:
#     print(corpus2id[corpus[tup[1]]])
query2doc

{0: ['202103191552312605.pdf.mr.txt',
  '202306301832142205.pdf.mr.txt',
  '202208171444300405.pdf.mr.txt',
  '202101141237329905.pdf.mr.txt',
  '202301101854070205.pdf.mr.txt',
  '202301231656422205.pdf.mr.txt',
  '202202041302069905.pdf.mr.txt',
  '202201251232153205.pdf.mr.txt',
  '202301101843336205.pdf.mr.txt'],
 1: ['202306061627330305.pdf.mr.txt',
  '202201061448221305.pdf.mr.txt',
  '202201031539413205.pdf.mr.txt',
  '202309051622471005.pdf.mr.txt',
  '202108041510112705.pdf.mr.txt',
  '202309061532428905.pdf.mr.txt',
  '202201141209026405.pdf.mr.txt',
  '202206081538326405.pdf.mr.txt',
  '202210031103191705.pdf.mr.txt'],
 2: ['202101291654193905.pdf.mr.txt',
  '202201181712239705.pdf.mr.txt',
  '202110111509425405.pdf.mr.txt',
  '202209201119052805.pdf.mr.txt',
  '202209301445033205.pdf.mr.txt',
  '202310111812522805.pdf.mr.txt',
  '202302081903148705.pdf.mr.txt',
  '202202031516434805.pdf.mr.txt',
  '202103151733244305.pdf.mr.txt'],
 3: ['202208171444300405.pdf.mr.txt',
  '20

In [27]:
list(corpus2id.items())[0]

('# Page 1 महाराष्ट्र राज्य लॉटरी “महाराष्ट्र गजलक्ष्मी सोम”, “महाराष्ट्र गजलक्ष्मी मंगळ”, “महाराष्ट्र गजलक्ष्मी बुध”, “महाराष्ट्र गजलक्ष्मी गुरू", “महाराष्ट्र गजलक्ष्मी शुक्र”, “महाराष्ट्र गजलक्ष्मी शनि”, व "महाराष्ट्र गजलक्ष्मी रवि” या साप्ताहिक सोडतींची बक्षिससंरचनेत बदल करणेबाबत. महाराष्ट्र शासन वित्त विभाग शासन निर्णय क्रमांकः मरालॉ २०२१/प्र.क्र. ६२/लॉटरी २ नवीन प्रशासन भवन, ८ वा मजला, मंत्रालयासमोर, मुंबई - ४०००३२. दिनांक :- ०८ ऑगस्ट, २०२२. वाचा : १. उपसंचालक (वित्त व लेखा), महाराष्ट्र राज्य लॉटरी कार्यालय, वाशी, नवीमुंबई यांचे पत्र क्र. उपसं (विवले)/मरालॉ/योजना/२२५५/२०२२, दि.२१/०७/२०२२. २. वित्त विभाग, शासन परिपत्रक क्रमांकः मरालॉ २०२१/प्र.क्र.८३/लॉटरी २, दि. २१ डिसेंबर, २०२१. शासन निर्णय : “महाराष्ट्र गजलक्ष्मी सोम”, “महाराष्ट्र गजलक्ष्मी मंगळ”, “महाराष्ट्र गजलक्ष्मी बुध”, “महाराष्ट्र गजलक्ष्मी गुरू”, “महाराष्ट्र गजलक्ष्मी शुक्र”, “महाराष्ट्र गजलक्ष्मी शनि” व “महाराष्ट्र गजलक्ष्मी रवि” या साप्ताहिक सोडती केंद्र शासनाचा लॉटरी (विनियमन) अधिनियम-१९९८ व लॉटरी (विनियमन) नियम २०१० अन

In [38]:
gold_docs = list(eval_df['Documents'])
cnt = 0
invalid_cnt = 0
for i in range(0,len(eval_df)):
        gold = set([doc.split(".")[0] for doc in gold_docs[i].split()])
        # gold = gold_docs[i].split()[0].split(".")[0]
        retrieval = set([doc.split(".")[0] for doc in query2doc[i]])
        # print(gold)
        # print(retrieval)
        if len(set(gold).intersection(set(retrieval))) > 0:
        # if gold in retrieval:
            cnt += 1
    # except:
    #     invalid_cnt += 1

print(cnt/len(eval_df))

0.5588235294117647
